In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import sys
import csv
import os
from sklearn import metrics
from dateutil.parser import parse
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%matplotlib inline

In [2]:
import numpy as np, pandas as pd, os
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
import random
import string

In [3]:
data_version_input = "01-10-2020"

In [4]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G"],
    
    "cat_str" : ["Field_3","Field_4", "Field_12","Field_36", "Field_38", "Field_54", "Field_62","Field_65", \
                 "Field_66", "info_social_sex", "brief", "gioiTinh"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh"],
    
    "text" : ["Field_18","currentLocationCity", "homeTownCity", "Field_48", "Field_49",\
             "currentLocationCountry", 'homeTownState','data.basic_info.locale', "currentLocationName", \
              'homeTownName', "homeTownCountry", "maCv", "currentLocationState", "Field_61", "Field_56",],
    
    "drop" : ["Field_45", "Field_34","Field_68", "diaChi", "Field_46", "Field_55"]
}

df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["text"], axis = 1)
df_transform = df_transform.drop(data_type["date"], axis = 1)
for feature in  data_type["cat_num"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
for feature in  data_type["cat_str"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
"""    
for feature in data_type["date"]:
    df_transform[feature] = df_transform[feature].map(lambda x: float(parse(str(x)).timestamp()) if type(x) is str else x)
    df_transform[feature] = df_transform[feature].fillna(df_transform[feature].mean())
"""

df_transform = df_transform.fillna(df_transform.mean())

## Dimensionality reduction
pca = PCA(n_components = 150)
sel = pca.fit(df_transform)
df_transform = pd.DataFrame(sel.transform(df_transform))

train_dev = df_transform.iloc[:len(df_train),:]
test = df_transform.iloc[len(df_train):,:]

scaler = MinMaxScaler()
scaler.fit(train_dev)
X_transform = scaler.transform(train_dev)
X_transform = pd.DataFrame(data = X_transform)
test_transform = scaler.transform(test)

train_dev = df_transform.iloc[:len(df_train),:]
test = df_transform.iloc[len(df_train):,:]
test = pd.DataFrame(data = test_transform)
train_dev = pd.concat([df_train.id, df_train.label, X_transform], axis = 1)
test = pd.concat([df_test.id, test], axis = 1)
train_dev.reset_index(drop=True,inplace=True)

In [6]:
result_str = ''.join(random.choice(string.ascii_letters) for i in range(5))
data_version = f"06-10-2020-{result_str.lower()}"
print(data_version)

06-10-2020-yztmh


In [7]:
train_dev

,id,label,0,1,2,3,4,5,6,7,...,140,141,142,143,144,145,146,147,148,149
0,0,1,0.072510,0.490859,0.143503,0.003926,0.133112,0.472262,0.013076,0.301383,...,0.091273,0.406273,0.202576,0.135555,0.369108,0.373441,0.408479,0.442561,0.355422,0.482421
1,1,0,0.072510,0.490859,0.122063,0.013701,0.429914,0.473105,0.013060,0.172737,...,0.093747,0.400825,0.205085,0.136621,0.367732,0.375472,0.409102,0.442355,0.350283,0.482281
2,2,0,0.072510,0.490859,0.168494,0.003263,0.811922,0.473534,0.013056,0.213784,...,0.092960,0.404848,0.201820,0.135077,0.368158,0.374996,0.411770,0.443500,0.349474,0.481064
3,3,0,0.072510,0.490859,0.122063,0.013701,0.429914,0.473081,0.012991,0.304379,...,0.092880,0.398885,0.203950,0.135368,0.368122,0.375492,0.411315,0.442929,0.350885,0.482159
4,4,1,0.072510,0.490859,0.122063,0.013701,0.429914,0.473081,0.012991,0.304379,...,0.092880,0.398885,0.203950,0.135368,0.368122,0.375492,0.411315,0.442929,0.350885,0.482159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53025,53025,0,0.000000,0.491596,0.051538,0.001081,0.601086,0.471712,0.012490,0.142846,...,0.091026,0.384911,0.199501,0.141551,0.371136,0.377645,0.411102,0.446228,0.354318,0.481830
53026,53026,0,0.000000,0.491596,0.122757,0.013226,0.428963,0.473037,0.012597,0.301179,...,0.092826,0.398116,0.204635,0.136636,0.368877,0.375620,0.410363,0.443359,0.350788,0.483140
53027,53027,0,0.126636,0.494164,0.134865,0.003353,0.609096,0.472876,0.011449,0.309198,...,0.102450,0.314018,0.233429,0.157912,0.361387,0.380930,0.413418,0.432915,0.347809,0.443992
53028,53028,0,0.072510,0.490859,0.122063,0.013701,0.429914,0.473081,0.012991,0.304383,...,0.093550,0.399761,0.204409,0.136792,0.367991,0.374969,0.410597,0.443094,0.350761,0.482152


In [8]:
test

,id,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,53030,0.000000,0.491596,0.051604,0.000603,0.897267,0.472090,0.012360,0.316045,0.478963,...,0.093261,0.389910,0.207744,0.136293,0.366821,0.378413,0.411019,0.445213,0.348312,0.479004
1,53031,0.000000,0.491596,0.036454,0.001599,0.118337,0.470823,0.012453,0.311364,0.480848,...,0.091921,0.400589,0.203662,0.135035,0.368370,0.375610,0.411150,0.440855,0.349891,0.484509
2,53032,0.072510,0.490859,0.122063,0.013701,0.429914,0.473081,0.012991,0.304382,0.454628,...,0.094668,0.398783,0.204349,0.135788,0.367595,0.375014,0.410678,0.443070,0.350881,0.482567
3,53033,0.124218,0.494115,0.050178,0.002669,0.111270,0.471053,0.012242,0.307519,0.079230,...,0.093544,0.396892,0.202919,0.137573,0.368028,0.378093,0.412473,0.441403,0.347292,0.483112
4,53034,0.000000,0.491596,0.122757,0.013226,0.428963,0.473042,0.012518,0.301885,0.460932,...,0.094880,0.399960,0.203572,0.136366,0.367986,0.375965,0.409580,0.441904,0.349908,0.483055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,0.072510,0.490859,0.122063,0.013701,0.429914,0.473081,0.012991,0.304388,0.454626,...,0.090782,0.398330,0.204046,0.135607,0.367745,0.374896,0.408923,0.442915,0.350898,0.482628
20377,73407,0.072510,0.490859,0.122063,0.013701,0.429914,0.473081,0.012991,0.304379,0.454629,...,0.092970,0.399105,0.204102,0.135760,0.368014,0.375610,0.411358,0.442984,0.350792,0.481915
20378,73408,0.126384,0.494159,0.206725,0.005327,0.160256,0.473179,0.011593,0.297425,0.522661,...,0.087677,0.404434,0.211473,0.140677,0.366365,0.376081,0.402552,0.441618,0.349560,0.480268
20379,73409,0.000000,0.491596,0.122757,0.013226,0.428963,0.473043,0.012504,0.302018,0.468953,...,0.091472,0.400905,0.203984,0.136385,0.367949,0.375212,0.408402,0.442044,0.351268,0.482328


In [9]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)